## Imports and Data Setup

In [1]:
# DISCLAIMER: For all feature engineering, I took the easy way out just to get my resampling and models 
# up and running. I'll be converting all my water column cleaning to functions, placing those functions 
# in functions.py, and then splitthing the model work from this notebook into syd_resampling.ipynb

In [2]:
# data
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import imblearn.under_sampling
import regex as re

# visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import seaborn as sns

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
data = pd.read_csv('../data/train_data.csv')
targets = pd.read_csv('../data/train_targets.csv')
test_data = pd.read_csv('../data/test_set_values.csv')

In [5]:
data = data.merge(targets, on='id', how='inner')

## Basic EDA

In [6]:
data.shape

(59400, 41)

In [7]:
data['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [8]:
data.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

#### Columns Addressed in this Notebook

amount_tsh - total static head
water_quality - The quality of the water
quality_group - The quality of the water
source - The source of the water
source_type - The source of the water
source_class - The source of the water
waterpoint_type - The kind of waterpoint
waterpoint_type_group - The kind of waterpoint

In [9]:
data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [10]:
data.drop('scheme_name', axis=1, inplace=True)
data.dropna(axis=0, how='any', inplace=True)

In [11]:
# data['amount_tsh'].value_counts()

In [12]:
# data.isnull().sum()
# no missing data in water columns

In [13]:
data['water_quality'].value_counts()

soft                  42667
salty                  3718
unknown                 880
coloured                320
milky                   299
salty abandoned         231
fluoride                160
fluoride abandoned       13
Name: water_quality, dtype: int64

In [14]:
data['quality_group'].value_counts()

good        42667
salty        3949
unknown       880
colored       320
milky         299
fluoride      173
Name: quality_group, dtype: int64

In [15]:
data['source'].value_counts()

spring                  14852
shallow well            12415
machine dbh              8730
river                    8506
rainwater harvesting     1607
hand dtw                  779
lake                      604
dam                       581
other                     182
unknown                    32
Name: source, dtype: int64

In [16]:
data['source_type'].value_counts()

spring                  14852
shallow well            12415
borehole                 9509
river/lake               9110
rainwater harvesting     1607
dam                       581
other                     214
Name: source_type, dtype: int64

In [17]:
data['source_class'].value_counts()

groundwater    36776
surface        11298
unknown          214
Name: source_class, dtype: int64

In [18]:
data['waterpoint_type'].value_counts()

communal standpipe             23837
hand pump                      13602
communal standpipe multiple     5459
other                           4651
improved spring                  651
cattle trough                     82
dam                                6
Name: waterpoint_type, dtype: int64

In [19]:
data['waterpoint_type_group'].value_counts()

communal standpipe    29296
hand pump             13602
other                  4651
improved spring         651
cattle trough            82
dam                       6
Name: waterpoint_type_group, dtype: int64

In [20]:
# data.dtypes

# amount_tsh - float64
# water_quality - object = 
# quality_group - object = 
# source - object = 
# source_type - object = 
# source_class - object = 
# waterpoint_type - object = 
# waterpoint_type_group - object = 

In [21]:
data.isnull().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
w

In [22]:
data.shape

(48288, 40)

In [23]:
# data.dtypes

## Non-water Columns Management

In [24]:
data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,GeoData Consultants Ltd,VWC,True,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,GeoData Consultants Ltd,VWC,True,0,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional


In [25]:
# good to go:

# id, amount_tsh, gps_height, population

In [26]:
# date_recorded AND construction_year

data['year'] = pd.DatetimeIndex(data['date_recorded']).year
data['age_in_years'] = data['year'] - data['construction_year']

data.drop(['date_recorded', 'year', 'construction_year'], axis=1, inplace=True)

In [27]:
# funder

data['funder'] = data['funder'].astype('category')
data['funder'] = data['funder'].cat.codes

In [28]:
# installer

data['installer'] = data['installer'].astype('category')
data['installer'] = data['installer'].cat.codes

In [29]:
# wtp_name

data.drop('wpt_name', axis=1, inplace=True)

In [30]:
# num_private has no descriptor in data source. since we don't know what it is, 
# we can't really derive insights from it. dropping.

data.drop('num_private', axis=1, inplace=True)

In [31]:
# basin

data['basin'] = data['basin'].astype('category')
data['basin'] = data['basin'].cat.codes

In [32]:
# subvillage - label encoding (too many for onehot or binary, still want to preserve unique identities)
# although I believe we can bin by distance/geographic region from Jacob's location work

data['subvillage'] = data['subvillage'].astype('category')
data['subvillage'] = data['subvillage'].cat.codes

In [33]:
# region - label encoding (too many for onehot or binary, still want to preserve unique identities)
# although I believe we can bin by distance/geographic region from Jacob's location work

data['region'] = data['region'].astype('category')
data['region'] = data['region'].cat.codes

In [34]:
# region code - drop because this is just a number and we want to use the label encoded 'region'

data.drop('region_code', axis=1, inplace=True)

In [35]:
# district_code - label encode

data['district_code'] = data['district_code'].astype('category')
data['district_code'] = data['district_code'].cat.codes

In [36]:
# lga - label encoding (too many for onehot or binary, still want to preserve unique identities)
# although I believe we can bin by distance/geographic region from Jacob's location work

data['lga'] = data['lga'].astype('category')
data['lga'] = data['lga'].cat.codes

In [37]:
# ward - label encoding (too many for onehot or binary, still want to preserve unique identities)
# although I believe we can bin by distance/geographic region from Jacob's location work

data['ward'] = data['ward'].astype('category')
data['ward'] = data['ward'].cat.codes

##### public meeting go to binary

In [38]:
data['public_meeting'] = data['public_meeting'].astype(int)

##### recorded_by - only has one value, GeoData Consultants Ltd. Dropping.

In [39]:
data.drop('recorded_by', axis=1, inplace=True)

##### scheme_management - label encode based on value counts, except move single None value into Other

In [40]:
data['scheme_management'].replace(to_replace='None', value='Other', inplace=True)

In [41]:
data['scheme_management'] = data['scheme_management'].astype('category')
data['scheme_management'] = data['scheme_management'].cat.codes

##### permit go to binary

In [42]:
data['permit'] = data['permit'].astype(int)

##### construction year: see data_recorded cell above

In [43]:
# extraction_type
# data['extraction_type'].value_counts()
# change other- rope pump to rope pump
# for now we will drop and just use extraction_type_class

data.drop('extraction_type', axis=1, inplace=True)

In [44]:
# extraction_type_group
# data['extraction_type_group'].value_counts()
# MISSING: ksb, swn 81, cemo, play pump, walimi, climax, mkuima/shinyanga
# change wind-powered to windmill
# for now we will drop and just use extraction_type_class

data.drop('extraction_type_group', axis=1, inplace=True)

In [45]:
# extraction_type_class compare to above 2, run a function to count rows where values 
# in these 3 are different from each other AFTER i've gone over options in each
# for now we do easy peasy encode

data['extraction_type_class'] = data['extraction_type_class'].astype('category')
data['extraction_type_class'] = data['extraction_type_class'].cat.codes

In [46]:
# management
# for now we drop, but we actually prefer this column over management_group, more info.

data.drop('management', axis=1, inplace=True)

In [47]:
# management_group - easy encode for now, but we will eventually drop this column in favor of management
data['management_group'] = data['management_group'].astype('category')
data['management_group'] = data['management_group'].cat.codes

In [48]:
# payment is same as payment_type, dropping
data.drop('payment', axis=1, inplace=True)

In [49]:
# payment_type - find someway to turn this into frequency amount, for now easy encode

data['payment_type'] = data['payment_type'].astype('category')
data['payment_type'] = data['payment_type'].cat.codes

In [50]:
data.head()

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region,district_code,lga,ward,population,public_meeting,scheme_management,permit,extraction_type_class,management_group,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,age_in_years
0,69572,6000.0,1131,1390,1254,34.938093,-9.856322,1,9941,3,5,47,1270,109,1,6,0,0,4,0,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,12
2,34310,25.0,696,686,1713,37.460664,-3.821329,5,7638,8,4,100,1455,250,1,6,1,0,4,5,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,4
3,67743,0.0,1449,263,1540,38.486161,-11.155298,7,7562,12,16,80,1408,58,1,6,1,5,4,2,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,27
5,9944,20.0,818,0,322,39.172796,-4.765587,5,9945,20,8,62,1180,1,1,6,1,5,4,5,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional,2
6,19816,0.0,288,0,333,33.362410,-3.766365,0,3165,17,3,96,1650,0,1,6,1,1,4,2,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,2012


## Water Columns Management

##### amount_tsh

In [51]:
# get rid of decimals

data['amount_tsh'] = data['amount_tsh'].astype(int)

##### water_quality AND quality_group

In [52]:
data['water_quality'].value_counts()

soft                  42667
salty                  3718
unknown                 880
coloured                320
milky                   299
salty abandoned         231
fluoride                160
fluoride abandoned       13
Name: water_quality, dtype: int64

In [53]:
data.drop('quality_group', axis=1, inplace=True)

In [75]:
data

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region,district_code,lga,ward,population,public_meeting,scheme_management,permit,extraction_type_class,management_group,payment_type,water_quality,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,age_in_years,salty_quality
0,69572,6000,1131,1390,1254,34.938093,-9.856322,1,9941,3,5,47,1270,109,1,6,0,0,4,0,soft,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,12,0
2,34310,25,696,686,1713,37.460664,-3.821329,5,7638,8,4,100,1455,250,1,6,1,0,4,5,soft,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,4,0
3,67743,0,1449,263,1540,38.486161,-11.155298,7,7562,12,16,80,1408,58,1,6,1,5,4,2,soft,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,27,0
5,9944,20,818,0,322,39.172796,-4.765587,5,9945,20,8,62,1180,1,1,6,1,5,4,5,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional,2,0
6,19816,0,288,0,333,33.362410,-3.766365,0,3165,17,3,96,1650,0,1,6,1,1,4,2,soft,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,2012,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394,11164,500,1558,351,864,37.634053,-6.124830,8,5898,11,6,76,191,89,1,6,1,5,4,1,soft,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,non functional,4,0
59395,60739,10,372,1210,163,37.169807,-3.253847,5,4736,6,5,14,967,125,1,9,1,0,4,5,soft,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,14,0
59396,27263,4700,140,1212,217,35.249991,-9.070629,6,2495,3,4,84,310,56,1,6,1,0,4,0,soft,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional,15,0
59398,31282,0,739,0,1006,35.861315,-6.378573,6,11780,2,4,9,1290,0,1,6,1,1,4,2,soft,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional,2011,0


In [78]:
data.columns

Index(['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'subvillage', 'region', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'scheme_management', 'permit',
       'extraction_type_class', 'management_group', 'payment_type',
       'water_quality', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group', 'age_in_years', 'salty_quality'],
      dtype='object')

In [92]:
# create the dictionary
#water_qualities = {'s'}  

data['salty_quality'] = 0
data['soft_quality'] = 0
data['color_quality'] = 0
data['opaque_quality'] = 0
data['flouride_quality'] = 0


def qualify_water(row):
    if row['water_quality'] == 'salty':
        return 1
    
        
#def qualify_water(data): 
 #   data['salt_quality'] = data['water_quality'].map(lambda x : 1 if x=='salty' else 0)
  #  data['soft_quality']=data['water_quality'].map(lambda x : 1 if x=='soft' else 0)
  #  data['color_quality']=data['water_quality'].map(lambda x : 1 if x=='coloured' else 0)
  #  data['opaque_quality']=data['water_quality'].map(lambda x : 1 if x=='milky' else 0)
  #  data['flouride_quality']=data['water_quality'].map(lambda x : 1 if x=='flouride' else 0)
  #  print(data)
    
# Define a function to map the values 
#def set_value(row_number, assigned_value): 
#    return assigned_value[row_number] 
  


#data['salty_quality'] = data['water_quality'].apply(qualify_water, args =(water_qualities, )) 
#data['soft_quality'] = data['water_quality'].apply(qualify_water, args =(water_qualities, )) 
#data['colored_quality'] = data['water_quality'].apply(qualify_water, args =(water_qualities, )) 
#data= data['water_quality'].apply(qualify_water, args =(water_qualities, )) 
#data['flouride_quality'] = data['water_quality'].apply(qualify_water, args =(water_qualities, )) 
  
# Print the DataFrame 
#print(df) 



#qualities = ['soft'. 'salty', 'unknown', 'coloured', 'milky',
 #           'salty abandoned', 'flouride', 'flouride abandoned']

# Create the dictionary 
#flouride_dict = {'flouride' : 1, 'flouride abandoned' : 0} 
#salty_dict = {'salty' : 1, 'salty abandoned' : 0}
#colored_dict = {'coloured' : 1, }
#opaque_dict = {'milky' : 1, }
#soft_dict = 
  
# Add a new column named 'Price' 
#df['Price'] = df['Event'].map(event_dictionary) 
  
# Print the DataFrame 
#print(df) 

In [94]:
data.apply(qualify_water)

KeyError: 'water_quality'

In [ ]:
# flouride to binary with flouride abandonded folded in
# salty to binary with salty abandoned folded in

##### quantity + quantity_group

In [ ]:
# quanity and quanity group go to scale of 1-5, delete one or keep both?

##### source

In [ ]:
# reference here: https://www.hrgreen.com/articles/choosing-right-water-storage-community-important-decision/

In [ ]:
data['source'].value_counts()

##### source_type

In [ ]:
data['source_type'].value_counts()

##### source_class

##### waterpoint_type

##### waterpoint_type_group

## Visualizations

## Notes

In [ ]:
# when stretching this, build a proposed schedule as business question that predicts how often various
# types of maintence are needed